In [1]:
from openai import OpenAI
import os 
import json
from datetime import datetime
from typing import List, Dict, Any
import IPython.display as display
from dotenv import load_dotenv
import sqlite3
import gradio as gr


In [2]:


load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4.1-mini"
BASE_URL = "https://openrouter.ai/api/v1"
openai = OpenAI(base_url=BASE_URL, api_key=openai_api_key)

In [3]:
system_prompt = """
You are a helpful assistant that provides information about the about grocery store and its products, be polite and courteous. 
You have access to the following information about the store and its products: price, availability, discounts, and location in the store.
if the user asks about a product that is not in the store, politely inform them that the product is not available and suggest similar products from the tools provided.
"""    

In [4]:
DB = 'grocery_store.db'
conn = sqlite3.connect(DB)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY, 
    name TEXT NOT NULL,
    price REAL NOT NULL,
    availability TEXT NOT NULL,
    discounts TEXT,
    category TEXT,
    location TEXT NOT NULL
)
''')

products = [
    (1, 'Milk', 2.99, 'In Stock', '10% off', 'Dairy', 'Aisle 1'),
    (2, 'Bread', 1.99, 'In Stock', '5% off', 'Bakery', 'Aisle 2'),
    (3, 'Eggs', 3.49, 'Out of Stock', None, 'Dairy', 'Aisle 3'),
    (4, 'Cheese', 4.99, 'In Stock', '15% off', 'Dairy', 'Aisle 4'),
    (5, 'Apples', 0.99, 'In Stock', None, 'Produce', 'Aisle 5'),
    (6, 'Bananas', 0.59, 'In Stock', '20% off', 'Produce', 'Aisle 6'),
    (7, 'Oranges', 1.29, 'In Stock', '10% off', 'Produce', 'Aisle 7'),
    (8, 'Tomatoes', 1.49, 'In Stock', '5% off', 'Produce', 'Aisle 8'),
    (9, 'Chicken Breast', 6.99, 'In Stock', '10% off', 'Meat', 'Aisle 9'),
    (10, 'Salmon Fillet', 9.99, 'In Stock', None, 'Seafood', 'Aisle 10'),
    (11, 'Spinach', 2.49, 'In Stock', '5% off', 'Produce', 'Aisle 11'),
    (12, 'Yogurt', 1.19, 'In Stock', '10% off', 'Dairy', 'Aisle 1'),
    (13, 'Butter', 2.79, 'In Stock', None, 'Dairy', 'Aisle 1'),
    (14, 'Carrots', 0.89, 'In Stock', None, 'Produce', 'Aisle 12'),
    (15, 'Potatoes', 0.69, 'In Stock', '5% off', 'Produce', 'Aisle 13'),
    (16, 'Ground Beef', 5.49, 'In Stock', '15% off', 'Meat', 'Aisle 9'),
    (17, 'Shrimp', 8.99, 'Out of Stock', None, 'Seafood', 'Aisle 10'),
    (18, 'Lettuce', 1.09, 'In Stock', None, 'Produce', 'Aisle 11'),
    (19, 'Cereal', 3.99, 'In Stock', '10% off', 'Grocery', 'Aisle 14'),
    (20, 'Orange Juice', 2.59, 'In Stock', '5% off', 'Beverages', 'Aisle 15')
]

cursor.executemany('INSERT OR IGNORE INTO products VALUES (?, ?, ?, ?, ?, ?, ?)', products)
conn.commit()


In [5]:

def query_product_info(product_name: str) -> Dict[str, Any]:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT name, price, availability, discounts, location FROM products WHERE name Like ?', (product_name,))
        result = cursor.fetchone()
        if result:
            return {
                'name': result[0],
                'price': result[1],
                'availability': result[2],
                'discounts': result[3],
                'location': result[4]
            }
        else:
            return None 

In [6]:
print(query_product_info(product_name='bread'))

{'name': 'Bread', 'price': 1.99, 'availability': 'In Stock', 'discounts': '5% off', 'location': 'Aisle 2'}


In [ ]:
def find_similar_products_from_category(category: str) -> List[Dict[str, Any]]:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT name, price, availability, discounts, location FROM products WHERE category = ?', (category,))
        results = cursor.fetchall()
        similar_products = []
        for result in results:
            similar_products.append({
                'name': result[0],
                'price': result[1],
                'availability': result[2],
                'discounts': result[3],
                'location': result[4]
            })
        print(results, 'resuults from database')
        return similar_products

In [8]:
availability = {
    "name": "query_product_info",
    "description": "Get the information of a product by its name.",
    "parameters": {
        "type": "object",
        "properties": {
            "product_name": {
                "type": "string",
                "description": "The name of the product to get information for",
            },
            "category": {
                "type": "string",
                "description": "The category of the product to get information for",
            },
        },
        "required": ["product_name"],
        "additionalProperties": False
    }
}

unAvailable = {
    "name": "find_similar_products_from_category",
    "description": "Get the information of a product by its name and get similar item for the search ptoduct.",
    "parameters": {
        "type": "object",
        "properties": {
            "category": {
                "type": "string",
                "description": "The category of the product to get information for",
            },
        },
        "required": ["category"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": availability}]
tools.append({"type": "function", "function": unAvailable})

Tool call back to check for the product and get the availability

In [9]:
def tools_callback(message: list):
    response = []
    for tools in message:
        if tools.function.name == 'query_product_info':
            print(f'tool call: {tools.function.name}')
            arguments = json.loads(tools.function.arguments)
            product = arguments.get('product_name')
            content = query_product_info(product_name=product)
            
        if tools.function.name == 'find_similar_products_from_category':
            print(f'tool call: {tools.function.name} args--> {tools.function.arguments}')
            arguments = json.loads(tools.function.arguments)
            category = arguments.get('category')
            content = find_similar_products_from_category(category=category)
            
        response.append({
            'role': 'tool',
            'content': json.dumps(content),
            "tool_call_id": tools.id,
            "name": tools.function.name,
        })
    return response

#UI code and the call to the function of the grocery store
The ai get the user infor mation and pass the data to check the local database for availability and return
the discounted price, location, etc.

In [29]:
def chat_with_store_assistant(user_input: str, history: dict) -> str:
    histories = [{"role": h['role'], 'content': h['content']} for h in history]
    messages = [{"role": 'system', 'content': system_prompt}] + histories + [{"role": "user", "content": user_input}]
    response = openai.chat.completions.create(
        model=MODEL,
        messages = messages,
        tools=tools
    )
    message = response.choices[0].message
    if message.tool_calls:
        messages.append(message)
        res = tools_callback(message.tool_calls)
        print('response from data --', type(res[0]))
        for i in messages:
            print(i)
        if res[0]['content'] != 'null':
            messages.append(res[0])

            content = openai.chat.completions.create(
                model=MODEL,
                messages=messages
            )
            message = content.choices[0].message.content
        
        else:
            message = "We do not have that product."    
        
        return message
        
    
gr.ChatInterface(fn=chat_with_store_assistant ).launch()

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


tool call: query_product_info
tool call: find_similar_products_from_category args--> {"category":"grocery"}
[] resuults from database
response from data -- <class 'dict'>
{'role': 'system', 'content': '\nYou are a helpful assistant that provides information about the about grocery store and its products, be polite and courteous. \nYou have access to the following information about the store and its products: price, availability, discounts, and location in the store.\nif the user asks about a product that is not in the store, politely inform them that the product is not available and suggest similar products from the tools provided.\n'}
{'role': 'user', 'content': 'salmon grape'}
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_zqY7Sk5dkxDxJYz2KlM2qQ1w', function=Function(arguments='{"product_name":"salmon grape","category":"grocery"}', name='query_product_info')